In [1]:
# !pip install scenedetect[opencv] --upgrade
# !pip install openai

In [1]:
from scenedetect import detect, ContentDetector
from __future__ import print_function
import os
import scenedetect
import pandas as pd

# Standard PySceneDetect imports:
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
# For caching detection metrics and saving/loading to a stats file
from scenedetect.stats_manager import StatsManager

# For content-aware scene detection:
from scenedetect.detectors.content_detector import ContentDetector


In [2]:
video_path = 'Episode 001.mp4'

scene_list = detect(video_path, ContentDetector())
scene_list

[(00:00:00.000 [frame=0, fps=24.997], 00:00:05.401 [frame=135, fps=24.997]),
 (00:00:05.401 [frame=135, fps=24.997], 00:00:35.364 [frame=884, fps=24.997]),
 (00:00:35.364 [frame=884, fps=24.997], 00:00:36.924 [frame=923, fps=24.997]),
 (00:00:36.924 [frame=923, fps=24.997], 00:00:37.685 [frame=942, fps=24.997]),
 (00:00:37.685 [frame=942, fps=24.997], 00:00:41.645 [frame=1041, fps=24.997]),
 (00:00:41.645 [frame=1041, fps=24.997],
  00:00:42.405 [frame=1060, fps=24.997]),
 (00:00:42.405 [frame=1060, fps=24.997],
  00:00:45.926 [frame=1148, fps=24.997]),
 (00:00:45.926 [frame=1148, fps=24.997],
  00:00:47.006 [frame=1175, fps=24.997]),
 (00:00:47.006 [frame=1175, fps=24.997],
  00:00:49.046 [frame=1226, fps=24.997]),
 (00:00:49.046 [frame=1226, fps=24.997],
  00:00:49.766 [frame=1244, fps=24.997]),
 (00:00:49.766 [frame=1244, fps=24.997],
  00:00:50.886 [frame=1272, fps=24.997]),
 (00:00:50.886 [frame=1272, fps=24.997],
  00:00:52.446 [frame=1311, fps=24.997]),
 (00:00:52.446 [frame=131

In [3]:
columns = ["Scene Number", "Starting Frame Time", "Starting Frame Number", "Ending Frame Time", "Ending Frame Number", "Duration of the Scene"]

# Initialize an empty dataframe
scene_df = pd.DataFrame(columns=columns)

# Iterate over each scene in the list
for i, scene in enumerate(scene_list):
    start_time = scene[0].get_timecode()
    start_frame = scene[0].get_frames()
    #fps = scene[0].get_fps()

    end_time = scene[1].get_timecode()
    end_frame = scene[1].get_frames()

    # Add a new row to the dataframe
    scene_df.loc[i] = [i+1, start_time, start_frame, end_time, end_frame, pd.to_datetime(end_time) - pd.to_datetime(start_time)]

scene_df.sort_values(by='Duration of the Scene', ascending = False).head(20)
scene_df


,Scene Number,Starting Frame Time,Starting Frame Number,Ending Frame Time,Ending Frame Number,Duration of the Scene
0,1,00:00:00.000,0,00:00:05.401,135,0 days 00:00:05.401000
1,2,00:00:05.401,135,00:00:35.364,884,0 days 00:00:29.963000
2,3,00:00:35.364,884,00:00:36.924,923,0 days 00:00:01.560000
3,4,00:00:36.924,923,00:00:37.685,942,0 days 00:00:00.761000
4,5,00:00:37.685,942,00:00:41.645,1041,0 days 00:00:03.960000
...,...,...,...,...,...,...
289,290,00:21:04.273,31603,00:21:08.274,31703,0 days 00:00:04.001000
290,291,00:21:08.274,31703,00:21:14.274,31853,0 days 00:00:06
291,292,00:21:14.274,31853,00:21:16.275,31903,0 days 00:00:02.001000
292,293,00:21:16.275,31903,00:21:18.275,31953,0 days 00:00:02


In [22]:
# def find_scenes(video_path):
#     # type: (str) -> List[Tuple[FrameTimecode, FrameTimecode]]
#     video_manager = VideoManager([video_path])
#     stats_manager = StatsManager()
#     # Construct our SceneManager and pass it our StatsManager.
#     scene_manager = SceneManager(stats_manager)

#     # Add ContentDetector algorithm (each detector's constructor
#     # takes detector options, e.g. threshold).
#     scene_manager.add_detector(ContentDetector())

#     # We save our stats file to {VIDEO_PATH}.stats.csv.
#     stats_file_path = '%s.stats.csv' % video_path

#     scene_list = []

#     try:
#         # If stats file exists, load it.
#         if os.path.exists(stats_file_path):
#             # Read stats from CSV file opened in read mode:
#             with open(stats_file_path, 'r') as stats_file:
#                 stats_manager.load_from_csv(stats_file)

#         # Set downscale factor to improve processing speed.
#         video_manager.set_downscale_factor()

#         # Start video_manager.
#         video_manager.start()

#         # Perform scene detection on video_manager.
#         scene_manager.detect_scenes(frame_source=video_manager)

#         # Obtain list of detected scenes.
#         scene_list = scene_manager.get_scene_list()
#         # Each scene is a tuple of (start, end) FrameTimecodes.

#         print('List of scenes obtained:')
#         for i, scene in enumerate(scene_list):
#             print(
#                 'Scene %2d: Start %s / Frame %d, End %s / Frame %d' % (
#                 i+1,
#                 scene[0].get_timecode(), scene[0].get_frames(),
#                 scene[1].get_timecode(), scene[1].get_frames(),))

#         # We only write to the stats file if a save is required:
#         if stats_manager.is_save_required():
#             base_timecode = video_manager.get_base_timecode()
#             with open(stats_file_path, 'w') as stats_file:
#                 stats_manager.save_to_csv(stats_file, base_timecode)

#     finally:
#         video_manager.release()

#     return scene_list

# find_scenes(video_path)

VideoManager is deprecated and will be removed.
base_timecode is deprecated.


List of scenes obtained:
Scene  1: Start 00:00:00.000 / Frame 0, End 00:00:05.040 / Frame 126
Scene  2: Start 00:00:05.040 / Frame 126, End 00:00:08.000 / Frame 200
Scene  3: Start 00:00:08.000 / Frame 200, End 00:00:10.000 / Frame 250
Scene  4: Start 00:00:10.000 / Frame 250, End 00:00:12.000 / Frame 300
Scene  5: Start 00:00:12.000 / Frame 300, End 00:00:14.000 / Frame 350
Scene  6: Start 00:00:14.000 / Frame 350, End 00:00:16.600 / Frame 415
Scene  7: Start 00:00:16.600 / Frame 415, End 00:00:18.520 / Frame 463
Scene  8: Start 00:00:18.520 / Frame 463, End 00:00:21.040 / Frame 526
Scene  9: Start 00:00:21.040 / Frame 526, End 00:00:26.000 / Frame 650
Scene 10: Start 00:00:26.000 / Frame 650, End 00:00:30.880 / Frame 772
Scene 11: Start 00:00:30.880 / Frame 772, End 00:00:33.160 / Frame 829
Scene 12: Start 00:00:33.160 / Frame 829, End 00:00:35.280 / Frame 882
Scene 13: Start 00:00:35.280 / Frame 882, End 00:00:40.520 / Frame 1013
Scene 14: Start 00:00:40.520 / Frame 1013, End 00:00:

[(00:00:00.000 [frame=0, fps=25.000], 00:00:05.040 [frame=126, fps=25.000]),
 (00:00:05.040 [frame=126, fps=25.000], 00:00:08.000 [frame=200, fps=25.000]),
 (00:00:08.000 [frame=200, fps=25.000], 00:00:10.000 [frame=250, fps=25.000]),
 (00:00:10.000 [frame=250, fps=25.000], 00:00:12.000 [frame=300, fps=25.000]),
 (00:00:12.000 [frame=300, fps=25.000], 00:00:14.000 [frame=350, fps=25.000]),
 (00:00:14.000 [frame=350, fps=25.000], 00:00:16.600 [frame=415, fps=25.000]),
 (00:00:16.600 [frame=415, fps=25.000], 00:00:18.520 [frame=463, fps=25.000]),
 (00:00:18.520 [frame=463, fps=25.000], 00:00:21.040 [frame=526, fps=25.000]),
 (00:00:21.040 [frame=526, fps=25.000], 00:00:26.000 [frame=650, fps=25.000]),
 (00:00:26.000 [frame=650, fps=25.000], 00:00:30.880 [frame=772, fps=25.000]),
 (00:00:30.880 [frame=772, fps=25.000], 00:00:33.160 [frame=829, fps=25.000]),
 (00:00:33.160 [frame=829, fps=25.000], 00:00:35.280 [frame=882, fps=25.000]),
 (00:00:35.280 [frame=882, fps=25.000], 00:00:40.520 [

In [25]:
from youtube_transcript_api import YouTubeTranscriptApi

YouTubeTranscriptApi.get_transcript('UUDKEbX5OQw')

[{'text': '[Music]', 'start': 0.0, 'duration': 15.16},
 {'text': 'chris gardner', 'start': 11.84, 'duration': 3.32},
 {'text': 'chris gardner', 'start': 16.16, 'duration': 3.119},
 {'text': 'how are you', 'start': 18.16, 'duration': 2.32},
 {'text': 'good morning', 'start': 19.279, 'duration': 3.201},
 {'text': 'chris gardner', 'start': 20.48, 'duration': 4.4},
 {'text': 'chris carter good to see you again',
  'start': 22.48,
  'duration': 5.36},
 {'text': 'chris gardner pleasure', 'start': 24.88, 'duration': 2.96},
 {'text': "i've been sitting out there for the last",
  'start': 28.48,
  'duration': 4.079},
 {'text': 'half hour trying to come up with a story',
  'start': 29.76,
  'duration': 5.12},
 {'text': 'that would explain my being here',
  'start': 32.559,
  'duration': 3.201},
 {'text': 'dressed', 'start': 34.88, 'duration': 2.24},
 {'text': 'like this', 'start': 35.76, 'duration': 3.68},
 {'text': 'and and i wanted to come up with a story',
  'start': 37.12,
  'duration': 4.24

In [7]:
from scenedetect import open_video

folder_name = video_path.split('.')[0]

if not os.path.exists(folder_name):
        os.mkdir(folder_name)
else:
    pass

video = open_video(video_path)

scenedetect.scene_manager.save_images(scene_list, video = video, output_dir = folder_name)

{0: ['Episode 001-Scene-001-01.jpg',
  'Episode 001-Scene-001-02.jpg',
  'Episode 001-Scene-001-03.jpg'],
 1: ['Episode 001-Scene-002-01.jpg',
  'Episode 001-Scene-002-02.jpg',
  'Episode 001-Scene-002-03.jpg'],
 2: ['Episode 001-Scene-003-01.jpg',
  'Episode 001-Scene-003-02.jpg',
  'Episode 001-Scene-003-03.jpg'],
 3: ['Episode 001-Scene-004-01.jpg',
  'Episode 001-Scene-004-02.jpg',
  'Episode 001-Scene-004-03.jpg'],
 4: ['Episode 001-Scene-005-01.jpg',
  'Episode 001-Scene-005-02.jpg',
  'Episode 001-Scene-005-03.jpg'],
 5: ['Episode 001-Scene-006-01.jpg',
  'Episode 001-Scene-006-02.jpg',
  'Episode 001-Scene-006-03.jpg'],
 6: ['Episode 001-Scene-007-01.jpg',
  'Episode 001-Scene-007-02.jpg',
  'Episode 001-Scene-007-03.jpg'],
 7: ['Episode 001-Scene-008-01.jpg',
  'Episode 001-Scene-008-02.jpg',
  'Episode 001-Scene-008-03.jpg'],
 8: ['Episode 001-Scene-009-01.jpg',
  'Episode 001-Scene-009-02.jpg',
  'Episode 001-Scene-009-03.jpg'],
 9: ['Episode 001-Scene-010-01.jpg',
  'Episod

['ojas']